# LightFM Hybrid Recommender

In [ ]:
!pip install -q implicit
!pip install -q lightfm

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.sparse import csr_matrix, coo_matrix
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

from metrics import precision_at_k as custom_precision, recall_at_k
from utils import prefilter_items

import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_parquet('./data/bronze/train.parquet.gzip')
item_features = pd.read_parquet('./data/bronze/products.parquet.gzip')
user_features = pd.read_parquet('./data/bronze/demographics.parquet.gzip')

In [ ]:
# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [ ]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [ ]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [ ]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


## Filter items

In [ ]:
n_items_before = data_train['item_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)

n_items_after = data_train_filtered['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 5001 to 2494


## Prepare data

In [ ]:
### Prepare csr train matrix
user_item_matrix = pd.pivot_table(data_train_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float)
sparse_user_item = csr_matrix(user_item_matrix).tocsr()
sparse_user_item

<2497x2494 sparse matrix of type '<class 'numpy.float64'>'
	with 226928 stored elements in Compressed Sparse Row format>

In [ ]:
### Prepare CSR test matrix
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float)
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [ ]:
### Prepare user and item features
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
user_feat.shape

(2497, 7)

In [ ]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)
item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
819255,1007.0,MEAT-PCKGD,National,BREAKFAST SAUSAGE/SANDWICHES,ROLLS - PORK,1 LB
819308,2879.0,MEAT,National,BEEF,CHOICE BEEF,


In [ ]:
item_feat.shape

(2494, 6)

## Encoding features

In [ ]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())
user_feat_lightfm.head(2)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,income_desc_125-149K,income_desc_15-24K,income_desc_150-174K,income_desc_175-199K,income_desc_200-249K,income_desc_25-34K,income_desc_250K+,income_desc_35-49K,income_desc_50-74K,income_desc_75-99K,income_desc_Under 15K,homeowner_desc_Homeowner,homeowner_desc_Probable Owner,homeowner_desc_Probable Renter,homeowner_desc_Renter,homeowner_desc_Unknown,hh_comp_desc_1 Adult Kids,hh_comp_desc_2 Adults Kids,hh_comp_desc_2 Adults No Kids,hh_comp_desc_Single Female,hh_comp_desc_Single Male,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Model training

In [ ]:
model = LightFM(no_components=40,
                loss='bpr', # "logistic","bpr"
                learning_rate=0.01, 
                item_alpha=0.4,
                user_alpha=0.1, 
                random_state=42,
                k=5,
                n=15,
                max_sampled=100)

In [ ]:
model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=20, 
          num_threads=20,
          verbose=True) 

Epoch: 100%|██████████| 20/20 [00:27<00:00,  1.36s/it]


## Getting embeddings

### Vectors by users

In [ ]:
user_emb = model.get_user_representations(features=csr_matrix(user_feat_lightfm.values).tocsr())
user_emb[0].shape # biases

(2497,)

In [ ]:
user_emb[1].shape # users vectors

(2497, 40)

### Vectors by products

In [ ]:
item_emb = model.get_item_representations(features=csr_matrix(item_feat_lightfm.values).tocsr())
item_emb[0].shape # biases

(2494,)

In [ ]:
item_emb[1].shape # items vectors

(2494, 40)

### Evaluation -> Train precision

In [ ]:
# we can use built-in lightFM metrics
train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

print(f"Train precision {train_precision}")

Train precision 0.2001602053642273


## Predict

In [ ]:
# prepare id for users and products in order of user-item pairs
users_ids_row = data_train_filtered['user_id'].apply(lambda x: userid_to_id[x]).values.astype(int)
items_ids_row = data_train_filtered['item_id'].apply(lambda x: itemid_to_id[x]).values.astype(int)
users_ids_row[:10]

array([2371, 1363, 1363, 1363, 1363, 1171, 1171, 1171, 1171, 1171])

In [ ]:
# the model returns a measure / rate of similarity between the corresponding user and the product
predictions = model.predict(user_ids=users_ids_row,
                            item_ids=items_ids_row,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=10)

# add to the train dataframe
data_train_filtered['score'] = predictions
data_train_filtered.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price,score
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0,2.99,-0.127823
11,1364,26984896261,1,842930,1,2.19,31742,0.00,1520,1,0.0,0.0,2.19,-2.048254
12,1364,26984896261,1,999999,1,2.99,31742,-0.40,1520,1,0.0,0.0,2.99,-1.937968
13,1364,26984896261,1,999999,1,3.09,31742,0.00,1520,1,0.0,0.0,3.09,-1.937968
14,1364,26984896261,1,999999,1,2.50,31742,-0.99,1520,1,0.0,0.0,2.50,-1.937968


In [ ]:
# create a predicate dataframe in list format
predict_result = data_train_filtered[['user_id','item_id','score']][data_train_filtered.item_id != 999999].drop_duplicates().sort_values(by=['user_id','score'], ascending=False).groupby('user_id')['item_id']. \
            unique().reset_index()
predict_result.head()

,user_id,item_id
0,1,"[1029743, 6034857, 1113385, 952163, 1004906, 9..."
1,2,"[1106523, 1075368, 969725, 899624, 952163, 904..."
2,3,"[1106523, 983584, 7025275, 5585510, 899624, 85..."
3,4,"[1029743, 1075368, 970760, 954966, 908283, 921..."
4,5,"[1126899, 1029743, 929018, 946995, 1050851, 94..."


In [ ]:
# combine the predicate and test dataset to calculate precision
df_result_for_metrics = result.merge(predict_result, on='user_id', how='inner')
df_result_for_metrics.head()

,user_id,actual,item_id
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1029743, 6034857, 1113385, 952163, 1004906, 9..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 983584, 7025275, 5585510, 899624, 85..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1070820, 1029743, 1126899, 9524291, 1121393, ..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1029743, 1126899, 1106523, 866540, 1093571, 1..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1106523, 1070820, 1029743, 872342, 5585510, 6..."


In [ ]:
# Test with custom precision func
precision = df_result_for_metrics.apply(lambda row: custom_precision(row['item_id'], row['actual'],k=5), axis=1).mean()
print(f"Precision: {precision}")

Precision: 0.1365169463639339


## Hyperparameter tuning

In [ ]:
# prepare id for users and products in order of user-item pairs
users_ids_row = data_train_filtered['user_id'].apply(lambda x: userid_to_id[x]).values.astype(int)
items_ids_row = data_train_filtered['item_id'].apply(lambda x: itemid_to_id[x]).values.astype(int)

In [ ]:
params_grid = {'item_alpha': [0.2, 0.4, 0.6],
               'user_alpha': [0.1, 0.2],
               'learning_rate' : [0.01, 0.05],
               'n': [10 , 100],
               'max_sampled': [10, 100]}

In [ ]:
result_searh = {}
index = 0

for item_alp in params_grid['item_alpha']:
    for user_alp in params_grid['n']:
        for learn_rate in params_grid['learning_rate']:
            for n in params_grid['n']:
                for max_sampl in params_grid['max_sampled']:

                    model = LightFM(no_components=40,
                          loss="bpr", # "logistic","bpr"
                          learning_rate=learn_rate, 
                          item_alpha=item_alp,
                          user_alpha=user_alp, 
                          random_state=42,
                          k=5,
                          n=n,
                          max_sampled=max_sampl)
                    model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
                    sample_weight=coo_matrix(user_item_matrix),
                    user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                    item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                    epochs=20, 
                    num_threads=20,
                    verbose=True) 

                    predictions = model.predict(user_ids=users_ids_row,
                                      item_ids=items_ids_row,
                                      user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                      item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                      num_threads=10)

                    data_train_filtered['score'] = predictions

                    predict_result = data_train_filtered[['user_id','item_id','score']][data_train_filtered.item_id != 999999].drop_duplicates().sort_values(by=['user_id','score'], ascending=False).groupby('user_id')['item_id']. \
                      unique().reset_index()

                    df_result_for_metrics = result.merge(predict_result, on='user_id', how='inner')

                    precision = df_result_for_metrics.apply(lambda row: custom_precision(row['item_id'], row['actual'],k=5), axis=1).mean()


                    result_searh[index] = [item_alp,
                                        user_alp,
                                        learn_rate,
                                        n,
                                        max_sampl,                                      
                                        df_result_for_metrics.apply(lambda row: custom_precision(row['item_id'], row['actual'],k=5), axis=1).mean()]
                    index += 1

In [ ]:
list(params_grid.keys())

['item_alpha', 'user_alpha', 'learning_rate', 'n', 'max_sampled']

In [ ]:
bpr_results = pd.DataFrame.from_dict(result_searh, orient='index',
                       columns=['item_alpha', 'user_alpha', 'learning_rate',
                                'n', 'max_sampled', 'score'])
bpr_results.sort_values(by='score', ascending=False).head()

,item_alpha,user_alpha,learning_rate,n,max_sampled,score
30,0.4,100,0.05,100,10,0.141848
31,0.4,100,0.05,100,100,0.137109
33,0.6,10,0.01,10,100,0.134049
18,0.4,10,0.01,100,10,0.128126
19,0.4,10,0.01,100,100,0.116379
